<a href="https://colab.research.google.com/github/iamdsc/humor_detection/blob/master/humor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Humor Detection in One-liners

In [0]:
## cloning the repo
!git clone https://github.com/iamdsc/humor_detection.git

Cloning into 'humor_detection'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 2), reused 9 (delta 2), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [0]:
%cd humor_detection/datasets/

/content/humor_detection/datasets


In [0]:
!ls

humorous_oneliners.pickle      proverbs.pickle		 wiki_sentences.pickle
oneliners_incl_doubles.pickle  reuters_headlines.pickle


In [0]:
import pandas as pd


humour = pd.read_pickle('humorous_oneliners.pickle')
proverb = pd.read_pickle('proverbs.pickle')
wiki = pd.read_pickle('wiki_sentences.pickle')
long_humour = pd.read_pickle('oneliners_incl_doubles.pickle')
reuters = pd.read_pickle('reuters_headlines.pickle')

In [0]:
print(len(humour), len(proverb), len(wiki), len(reuters))

5251 1019 5251 5243


In [0]:
# building dataframe from the data
# giving humourous sentence positive class and rest negative class
humour_record = [(sent, 1) for sent in humour]
proverb_record = [(sent, 0) for sent in proverb]
wiki_record = [(sent, 0) for sent in wiki]
reuter_record = [(sent, 0) for sent in reuters]

In [0]:
import random


pos_record = humour_record
neg_record = proverb_record + wiki_record + reuter_record
columns = ['sentence', 'class']

# Randomly picking len(pos_records) records from negative class
random.shuffle(neg_record)
neg_record = neg_record[:len(pos_record)]

# Build the dataframe
df_record = pos_record + neg_record 
df = pd.DataFrame(df_record, columns=columns)

# shuffle the dataframe and reset the index
df = df.sample(frac=1).reset_index(drop=True)
print(df.head())

                                            sentence  class
0                       A Black Hen Lays A White Egg      0
1  At what point does a Lamb become a Sheep? When...      1
2     U.S. jobless claims fall more than expected:        0
3  Periodically he would organize a new band and ...      0
4  Need experience for job. Need job for experience.      1


In [0]:
df['class'].value_counts()

1    5251
0    5251
Name: class, dtype: int64

In [0]:
# Creating word vectors
from gensim.models import Word2Vec

text=[]
for sentence,_ in df_record:
  sent_word_list = [word for word in sentence.lower().split()]
  text.append(sent_word_list)

w2v = Word2Vec(text, min_count=1)

In [0]:
# building list of sentence vectors
vect_record=[]
for i in range(len(df['sentence'])):
  sent = df['sentence'][i]
  if len(sent)!=0:
    sent_vect = sum([w2v[w] for w in sent.lower().split()])/(len(sent.split())+0.001)
  else:  
    sent_vect = np.zeros((100,))
  vect_record.append(sent_vect) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [0]:
X = pd.DataFrame(vect_record, columns=range(100))
print(X.head())
print(X.shape)

         0         1         2         3         4         5         6   \
0 -0.299317  0.261278  0.500360  0.514710  0.133264  0.317683 -0.058355   
1 -0.418768  0.344506  0.702678  0.761039  0.210620  0.422604 -0.046371   
2 -0.252735  0.216322  0.406221  0.424718  0.103921  0.253117 -0.037577   
3 -0.381051  0.328394  0.609629  0.633020  0.158734  0.391795 -0.073301   
4 -0.259524  0.233595  0.414892  0.419551  0.096390  0.273825 -0.052584   

         7         8         9     ...           90        91        92  \
0 -0.000443  0.297339 -0.935085    ...    -0.203419 -0.241452 -0.787392   
1  0.009775  0.351279 -1.235259    ...    -0.336546 -0.246760 -1.086021   
2 -0.002454  0.238438 -0.757676    ...    -0.170742 -0.188063 -0.645034   
3  0.000465  0.380359 -1.165326    ...    -0.241877 -0.314932 -0.983472   
4 -0.007803  0.281696 -0.810582    ...    -0.149636 -0.245752 -0.679057   

         93        94        95        96        97        98        99  
0  0.557964  0.159228 -0

In [0]:
y = df['class']
print(type(y))

<class 'pandas.core.series.Series'>


In [0]:
# Performaing train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [0]:
print('Training Data: ', X_train.shape, y_train.shape)
print('Test Data: ', X_test.shape, y_test.shape)

Training Data:  (8401, 100) (8401,)
Test Data:  (2101, 100) (2101,)


In [0]:
from sklearn.svm import SVC

# fit the SVM model with linear Kernel
svm = SVC(kernel='linear', random_state=1, C=1.0)
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=1,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
# performing prediction in test samples
y_pred = svm.predict(X_test)
print(y_pred[:5])

[0 1 0 0 0]


In [0]:
# Calculating Classification Accuracy
from sklearn.metrics import accuracy_score

print('Test Accuracy Score: ',accuracy_score(y_pred, y_test))

Test Accuracy Score:  0.817705854355069


In [0]:
# Calculating Cross Validation Accuracy
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=10, n_jobs=-1)
print('CV accuracy: %.3f +/- %.3f'%(np.mean(scores),np.std(scores)))

CV accuracy: 0.809 +/- 0.011


In [0]:
# Using Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train, y_train)

# performing prediction on test samples
y_pred = lr.predict(X_test)

print('Test Accuracy score: ', accuracy_score(y_pred, y_test))

scores = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=10, n_jobs=-1)
print('CV accuracy: %.3f +/- %.3f'%(np.mean(scores),np.std(scores)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test Accuracy score:  0.8810090433127082
CV accuracy: 0.872 +/- 0.007
